<a href="https://colab.research.google.com/github/Prajwal270/CNN1.1/blob/master/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print('Working with Transfer Learning')

Working with Transfer Learning


In [0]:
import tensorflow as tf


In [0]:
print(tf.__version__)

1.15.0


In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [0]:
import os

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-02-17 16:32:51--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   236MB/s    in 0.4s    

2020-02-17 16:32:51 (236 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [0]:
local_weight_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top = False,
                                weights = None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
pre_trained_model.load_weights(local_weight_file)

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [0]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [0]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [0]:
print('last layer output shape: ', last_layer.output_shape)

last layer output shape:  (None, 7, 7, 768)


In [0]:
from tensorflow.keras.optimizers import RMSprop

In [0]:
# Flatten the last layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1024 neuron and a relu activation function

x = layers.Dense(1024, activation='relu')(x)

#Add a dropout layer with 0.2 
x = layers.Dropout(0.2)(x)

# Add a Dense layer to do classification 

x = layers.Dense(1, activation ='sigmoid')(x)

In [0]:
model = Model(pre_trained_model.input, x)

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=RMSprop(lr = 0.005), metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Downloading out dataset as zip file

In [0]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

--2020-02-17 17:09:21--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  95.0MB/s    in 0.7s    

2020-02-17 17:09:22 (95.0 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
import os 
import zipfile

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
local_dir = '/tmp/cats_and_dogs_filtered.zip'

In [0]:
local_dir

'/tmp/cats_and_dogs_filtered.zip'

In [0]:
local_zip = '//tmp/cats_and_dogs_filtered.zip'

In [0]:
zip_ref = zipfile.ZipFile(local_zip, 'r')

In [0]:
zip_ref.extractall('/tmp')

In [0]:
zip_ref.close()

In [0]:
train_datagen = ImageDataGenerator(
    rescale = 1./225.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [0]:
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255.)

In [0]:
# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

In [0]:
train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

In [0]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
history = model.fit_generator(
    train_generator, 
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 20, 
    validation_steps = 50,
     verbose = 2
)

Epoch 1/20
Epoch 1/20
100/100 - 194s - loss: 4.4865 - acc: 0.6875 - val_loss: 0.4530 - val_acc: 0.9070
Epoch 2/20
Epoch 1/20
100/100 - 191s - loss: 0.6071 - acc: 0.7710 - val_loss: 0.3245 - val_acc: 0.9290
Epoch 3/20
Epoch 1/20
100/100 - 190s - loss: 0.6082 - acc: 0.7880 - val_loss: 0.3481 - val_acc: 0.9380
Epoch 4/20
Epoch 1/20
100/100 - 190s - loss: 0.4812 - acc: 0.8115 - val_loss: 0.2835 - val_acc: 0.9550
Epoch 5/20
Epoch 1/20
100/100 - 191s - loss: 0.4925 - acc: 0.8210 - val_loss: 1.1072 - val_acc: 0.8940
Epoch 6/20
Epoch 1/20
100/100 - 190s - loss: 0.4558 - acc: 0.8310 - val_loss: 0.3410 - val_acc: 0.9570
Epoch 7/20
Epoch 1/20
100/100 - 190s - loss: 0.4229 - acc: 0.8475 - val_loss: 0.3775 - val_acc: 0.9490
Epoch 8/20
Epoch 1/20
100/100 - 189s - loss: 0.4223 - acc: 0.8455 - val_loss: 0.3886 - val_acc: 0.9650
Epoch 9/20
Epoch 1/20
100/100 - 192s - loss: 0.4252 - acc: 0.8395 - val_loss: 1.2131 - val_acc: 0.9120
Epoch 10/20
Epoch 1/20
100/100 - 190s - loss: 0.4019 - acc: 0.8685 - val_